In [85]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models # models是一些pytorch基本模型的导入
from torch.utils.data import DataLoader 
import sys


In [86]:
# 数据预处理，对数据进行一些预处理的操作（裁剪，旋转等）
transform = transforms.Compose([
    transforms.RandomResizedCrop(224), # 对图像随机裁剪 变成224*224的图片
    transforms.RandomRotation(20),     # 对图像随机旋转
    transforms.RandomHorizontalFlip(p=0.5), # 对图像随机翻转
    transforms.ToTensor(), # 变成长度224的tensor
])

# 读取数据
root = 'image'
train_dataset = datasets.ImageFolder(root + '/train', transform)
test_dataset = datasets.ImageFolder(root + '/test', transform)

# 导入数据
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True) # 这里的batch_size指的是一个batch内东西的大小
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size= 8, shuffle = True)


print(len(train_dataset))
for i,data in enumerate(train_loader): #enumerate是为了每一批次的数据增加一个索引i
    inputs, labels = data # data里面包含了要输入的量和输出的标签值
    print(inputs.shape)
    print(labels.shape)
    break

400
torch.Size([8, 3, 224, 224])
torch.Size([8])


In [87]:
# 查看数据集属性
classes = train_dataset.classes
classes_index = train_dataset.class_to_idx
print(classes)
print(classes_index) 

['cat', 'dog']
{'cat': 0, 'dog': 1}


In [88]:
model = models.vgg16(pretrained = True) # 一般会使用已经训练好的模型再进行全连接层调参，这个是有1000个分类的
print(model) 

c:\Users\18103\anaconda3\envs\{AI_Training_Test}\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18103\anaconda3\envs\{AI_Training_Test}\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [89]:
# 仅训练模型的全连接层（使用别人已训练好的模型）
for param in model.parameters(): # 循环参数，设置梯度计算，看是否参与梯度计算，False就是不参与
    param.requires_grad = False

# 构建新的全连接层（只有这里参与训练）
# 直接在已有的模型里面把 模型定义的 classifier部分给改过来
model.classifier = nn.Sequential(torch.nn.Linear(25088, 100),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(p=0.5),
                                 torch.nn.Linear(100, 2))
                                              

In [90]:
# 测试是否有可用的GPU和CPU
# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 把模型转到GPU
model.to(device)   # 将模型移动到 GPU 或 CPU

Using device: cuda


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [91]:

entropy_loss = nn.CrossEntropyLoss() # 定义损失函数，可以直接调用entropy_loss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # 定义优化器

In [92]:
# 定义训练train() 和 test()
def train():
    model.train() # 训练模式，这个时候dropout是不启用的
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) # 转移到GPU
        out = model(inputs)
        loss = entropy_loss(out, labels) # 损失会自动到别的地方去的，因为前面用的是nn.CrossEntropyLoss()，是torch里面的函数

        optimizer.zero_grad() # 梯度清零
        loss.backward() # 反向传播
        optimizer.step() # 优化器一步

def test():
    model.eval() # 测试模式是用eval()函数！
    correct = 0
    for i, data in enumerate(test_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) # 转移到GPU
        out = model(inputs)
        _, predicted = torch.max(out.data, 1) # 取出每一行的最大值，和其所在的位置
        correct += (predicted == labels).sum()
    print("Test acc: {0}" .format(correct.item() /len (test_dataset)))

    correct = 0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) # 转移到GPU
        out = model(inputs)
        _, predicted = torch.max(out.data, 1) # 取出每一行的最大值，和其所在的位置
        correct += (predicted == labels).sum()
    print("Train acc: {0}" .format(correct.item() /len (train_dataset)))



In [93]:
for epoch in range(10):
    print("Epoch: ", epoch)
    train()
    test()

torch.save(model.state_dict(), 'cat_dog.pth') # 保存模型参数

Epoch:  0
Test acc: 0.85
Train acc: 0.825
Epoch:  1
Test acc: 0.855
Train acc: 0.895
Epoch:  2
Test acc: 0.82
Train acc: 0.8525
Epoch:  3
Test acc: 0.83
Train acc: 0.8825
Epoch:  4
Test acc: 0.865
Train acc: 0.8725
Epoch:  5
Test acc: 0.885
Train acc: 0.895
Epoch:  6
Test acc: 0.845
Train acc: 0.885
Epoch:  7
Test acc: 0.855
Train acc: 0.9
Epoch:  8
Test acc: 0.835
Train acc: 0.87
Epoch:  9
Test acc: 0.85
Train acc: 0.8825
